In [11]:
import json
import glob
import pandas as pd
import numpy as np
import sklearn
from numpy import asarray
from PIL import Image
import torch
from torchvision.transforms import ToTensor
from torch import flatten
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = '/Users/paddy/Desktop/AiCore/facebook_ml/final_dataset/image_data.csv'
images = '/Users/paddy/Desktop/AiCore/facebook_ml/images_raw'
images_corrected = '/Users/paddy/Desktop/AiCore/facebook_ml/Images'

In [2]:
image_list = glob.glob(f'{images}/*.jpg')
df = pd.read_csv(data)
# df = df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)

df

,Unnamed: 0,uuid,id,bucket_link,image_ref,create_time,price,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,id.1
0,0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,koQAAOSwhZRiE5dD,2022-02-26,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,243809c0-9cfc-4486-ad12-3b7a16605ba9
1,1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,9vgAAOSw8sRiE5dE,2022-02-26,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,dcbe214e-cb4b-4f18-ab16-218ec12ab68b
2,2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,FlwAAOSwCkpiFNlV,2022-02-26,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6f228cd8-a2d6-4f23-858b-4a77a2f42d7c
3,3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,4gIAAOSw4ORiFNlZ,2022-02-26,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2275f5f2-ee51-410c-8d85-e5923285a864
4,4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,https://aicore-product-images.s3.amazonaws.com...,sHoAAOSwQoRiD7vf,2022-02-26,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,c0e5ef79-09c0-4cb7-9b7c-edd3d53aa530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,6306,e2d7e9e4-8a94-4293-a7a4-e4209e3c9243,e95a25e7-f2be-4ca0-aabb-a3a3642d2cc6,https://aicore-product-images.s3.amazonaws.com...,8AEAAOSwTvBiGSwG,2022-02-27,55.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,a9e31ee1-c706-42a9-97d9-4ca257e4c6f3
6307,6307,4efda54a-d4e6-4d4f-8d59-dc1d9a3bbd58,0276f1d7-570a-4420-832a-d1a13c487c0a,https://aicore-product-images.s3.amazonaws.com...,6ycAAOSwSURgJ-Ba,2022-02-27,88.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,f3bcf410-3972-4021-bc0f-dae1892f8793
6308,6308,4be61860-85ad-4217-aacb-a3729176cad0,0276f1d7-570a-4420-832a-d1a13c487c0a,https://aicore-product-images.s3.amazonaws.com...,2AsAAOSwYoxgJ-Bc,2022-02-27,63.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,dd1a90ff-4a1f-4ff8-8183-381d79447bf2
6309,6309,53e142a5-b293-47a0-aff8-cd0db1cd291f,5083e833-e273-4ca9-b68f-a3cbb1bfeaf2,https://aicore-product-images.s3.amazonaws.com...,2i4AAOSw-r9iGSo3,2022-02-27,179.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ebe8d613-4aa0-477c-8f1a-256ff8d4f248


In [ ]:
df = df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)
df_for_model = df.drop(['uuid', 'price', 'id.1'], axis=1)
df_for_model

In [ ]:
# df.columns
column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
new_column = ['0','1','2','3','4','5','6','7','8','9','10','11']
old_names = df_for_model.columns[column_indicies]
df_for_model.rename(columns=dict(zip(old_names, new_column)),inplace=True)
df_for_model = df_for_model.drop(['id'], axis=1)

df_for_model

In [ ]:
# df_pls_work = df_for_model[df_for_model != 0].stack().reset_index(level=0, drop=True)
# print(df_for_model.columns[df_for_model == 1.0])
df_for_model['new'] = df_for_model.apply(lambda x: x.index[x == 1].tolist(), axis=1)
# print(df_for_model)
# df_for_model

class_list = df_for_model['new'].tolist()


In [ ]:
flat_list = []
for sublist in class_list:
    for item in sublist:
        flat_list.append(item)
print(len(flat_list))

final_cat_list = list(map(int, flat_list))
print(len(final_cat_list))

In [10]:
class image_setup:
    def __init__(self, df: pd):
        self.df = df
        self.image_list_fixed = glob.glob(f'{images_corrected}/*.jpg')
        self.image_id_list = []
        self.class_list = []
        self.flattened = []
        self.list_of_tuples = []
        self.df_cleaned(df)
        self.classification_names_list(df)
        self.image_correct_prop()
        self.image_to_tensor()
        self.training_tuples()

    
    def df_cleaned(self, df: pd):
        self.df = self.df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)
        self.df = self.df.drop(['uuid', 'price', 'id.1'], axis=1)
        column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
        classification_columns = ['0','1','2','3','4','5','6','7','8','9','10','11']
        old_names = self.df.columns[column_indicies]
        self.df.rename(columns=dict(zip(old_names, classification_columns)),inplace=True)
        self.df = self.df.drop(['id'], axis=1)


    def classification_names_list(self, df: pd):
        self.df['new'] = self.df.apply(lambda x: x.index[x == 1].tolist(), axis=1)
        flat_list = []
        class_list = self.df['new'].tolist()
        for sublist in class_list:
            for item in sublist:
                flat_list.append(item)
            self.class_list = list(map(int, flat_list))


    def image_correct_prop(self):
        resized_img = 128
        for n, img in enumerate(image_list[:50], 1):
            image_ite = Image.open(img)
            background = Image.new(mode='RGB', size=(resized_img, resized_img)) 
            original_img = image_ite.size
            max_dim = max(image_ite.size)
            ratio = resized_img / max_dim
            img_corr_ratio =(int(original_img[0] * ratio), int(original_img[1] * ratio))
            img_corr = image_ite.resize(img_corr_ratio)
            # print(img_corr)
            background.paste(img_corr, (((resized_img - img_corr_ratio[0]) // 2), ((resized_img - img_corr_ratio[1]) // 2)))
            # print(background)
            background.save(f'{images_corrected}/{img[50:]}')

    
    def image_to_tensor(self):
        for i, img in enumerate(self.image_list_fixed[:50]):
            image = Image.open(img)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(self.flattened)
            self.flattened.append(flatten_numpy)

        return self.flattened

    def training_tuples(self):
        for i in range(len(self.flattened)):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (self.flattened[i], self.class_list[i])
                self.list_of_tuples.append(ready_tuple)
                print(self.list_of_tuples)
        return self.list_of_tuples
        


            
    
    pass
image_setup(df)

[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)]
[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([1., 1., 1., ..., 1., 1., 1.], dtype=float32), 5)]
[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([1., 1., 1., ..., 1., 1., 1.], dtype=float32), 5), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)]
[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([1., 1., 1., ..., 1., 1., 1.], dtype=float32), 5), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([0.        , 0.89411765, 0.88235295, ..., 0.5647059 , 0.5529412 ,
       0.        ], dtype=float32), 5)]
[(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([1., 1., 1., ..., 1., 1., 1.], dtype=float32), 5), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5), (array([0.        , 0.89411765, 0.88235295, ..., 0.5647059 , 0.5529412 ,
       0.        ], dtype=float32), 5), (array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 5)]
[(array([0., 0.

In [ ]:
image_array = []
image_list_f = glob.glob(f'{images_corrected}/*.jpg')
def image_to_tensor():
        for img in image_list_f:
            image = Image.open(img)
            # print(image)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(flattened.dtype)
            image_array.append(flatten_numpy)

            # print(empty)
        return image_array
image_to_tensor()
print(len(image_array))


In [ ]:
list_of_tuples = []
# final_cat_list

print(len(final_cat_list))
def training_tuples():
        for i in range(6266):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (image_array[i], final_cat_list[i])
                list_of_tuples.append(ready_tuple)
        return list_of_tuples
training_tuples()
print(len(list_of_tuples))
# list_of_tuples[:5].shape

In [ ]:
print(list_of_tuples[1])

In [ ]:
print(image_array[1].shape)
size = 128*128*3
n = len(image_array)
print(len(image_array))
print(image_array[1])


In [12]:
# m = no of labels
#  features = 49152 = items in list of flattened image :)
m = len(list_of_tuples)
sh = image_array[0].shape
array_size = 49152
list_of_tuples
print(len(list_of_tuples))
X = np.zeros((m, array_size))
y = np.zeros(m)
print(X.shape, y.shape)
for item in range(m):
    features, labels = list_of_tuples[item]
    # print(features.shape)
    X[item, :] = features
    y[item] = labels
    # print(X)
    print(y)

NameError: name 'list_of_tuples' is not defined

In [ ]:
model = LogisticRegression(max_iter= 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print ('acc:', accuracy_score(y_test, pred))
print('prec:', classification_report(y_test, pred))